In [22]:
!pip install pyspark

In [23]:
!pip install findspark

In [24]:
# Import findspark and initialize.
import findspark
findspark.init()

In [25]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [27]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homesales')


In [28]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""
  SELECT
    YEAR(date) AS YEAR,
    ROUND(AVG(price),2) AS average_price
  FROM homesales
  WHERE bedrooms = 4
  GROUP BY YEAR
  """).show()


+----+-------------+
|YEAR|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [29]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
  SELECT
    date_built,
    ROUND(AVG(price),2) AS average_price
  FROM homesales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
  """).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [30]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
  SELECT
    date_built,
    ROUND(AVG(price),2) AS average_price
  FROM homesales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living > 2000
  GROUP BY date_built
  """).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    297619.46|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280527.62|
+----------+-------------+



In [31]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
  SELECT view, ROUND(AVG(price),2) AS average_price
  FROM homesales
  GROUP BY view
  HAVING AVG(price) >= 350000
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 0.8072996139526367 seconds ---


In [32]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homesales")

DataFrame[]

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('homesales')

True

In [34]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
  SELECT view
  FROM homesales
  GROUP BY view
  HAVING AVG(price) >= 350000
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+
|view|
+----+
|  85|
|  65|
|  53|
|  78|
|  81|
|  76|
|  91|
|  93|
|  52|
|  86|
|  94|
|  57|
|  54|
|  96|
|  92|
|  64|
|  61|
|  88|
|  72|
|  59|
+----+
only showing top 20 rows

--- 0.7004647254943848 seconds ---


In [35]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('parquet_delay_basic', mode='overwrite')

In [36]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parquet_delay_basic')

In [37]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('parquet_delay_basic')

In [38]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
  SELECT view, ROUND(AVG(price),2) AS average_price
  FROM parquet_delay_basic
  GROUP BY view
  HAVING AVG(price) >= 350000
  """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 1.4128532409667969 seconds ---


In [39]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homesales")

DataFrame[]

In [40]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('homesales')


False